In [19]:
from preprocesamiento import training_seq, SEQ_LEN
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

#es el tamaño del diccionario
OUTPUT_UNITS = 38
NUM_UNITS = 256
LOSS =  nn.CrossEntropyLoss()
LR = 0.01
EPOCH = 10
BATCH_SIZE = 64*2
SAVE_MODEL_PATH = 'model.pt'

#clase para generar una red lstm
class model_lstm(nn.Module):
    def __init__(self, output_units, num_units):
        super(model_lstm, self).__init__()
        self.lstm = nn.LSTM(output_units, num_units)
        self.dropout = nn.Dropout(0.1)
        self.linear = nn.Linear(num_units, output_units)

    def feed_forward(self, x):
        output, _ = self.lstm(x)
        output = self.dropout(output)
        output = self.linear(output)

        return output

def train(output_units = OUTPUT_UNITS, num_units = NUM_UNITS, loss = LOSS, lr = LR):
    inputs, targets = training_seq(SEQ_LEN)
    #los pasamos a tensores
    inputs = torch.from_numpy(inputs).float()
    targets = torch.from_numpy(np.array(targets)).long()

    #creamos la red y el optimizador
    red_lstm = model_lstm(output_units, num_units)
    optimizer = optim.Adam(red_lstm.parameters(), lr = lr)

    #entrenamos
    for epoch in range(EPOCH):
        loss_total = 0

        #mini batch
        for i in range(0, len(inputs), BATCH_SIZE):           
            batch_inputs = inputs[i:i+BATCH_SIZE]
            batch_targets = targets[i:i+BATCH_SIZE]
            
            print(f'batch_inputs = {batch_inputs.shape}')
            print(f'batch_targets = {batch_targets.shape}')

            optimizer.zero_grad()

            #hacemos feed forward
            outputs = red_lstm.feed_forward(batch_inputs)
            print(f'outputs: {outputs.shape}')

            loss_val = loss(outputs.permute(0, 2, 1), batch_targets)

            #backpropagation
            loss_val.backward()
            optimizer.step()

            loss_total += loss_val.item()

        print(f'Epoch: {epoch + 1} \t Loss: {loss_total:.5f}')

    #guardamos el modelo
    torch.save(red_lstm.state_dict(), SAVE_MODEL_PATH)
    #guardamos el modelo
    torch.save(red_lstm.state_dict(), SAVE_MODEL_PATH)

In [20]:
train()

batch_inputs = torch.Size([128, 64, 38])
batch_targets = torch.Size([128])
outputs: torch.Size([128, 64, 38])


RuntimeError: Expected target size [128, 64], got [128]